# Coursera Capstone Project
## The Battle of Neighborhoods (Week 1)

In [7]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Nageswar\Anaconda3

  added / updated specs:
    - geopy


The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.1-py37_0 --> conda-forge::conda-4.8.2-py37_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [8]:
# A description of the problem and a discussion of the background. (15 marks)

1. Introduction Section :
Discussion of the business problem and the audience who would be interested in this project.


# Description of the Problem and Background
## Scenario:
I am a data scientist residing in Downtown Singapore. I currently live within walking distance to Downtown Telok Ayer MRT metro station and I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment. I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How can I find a convenient and enjoyable place similar to mine now in Singapore? Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:
* apartment must be 2 or 3 bedrooms
* desired location is near a metro station in the Manhattan area and within 1.0 mile (1.6 km) radius
* price of rent not exceed $7,000 per month
* top ammenities in the selected neighborhood shall be similar to current residence
* desirable to have venues such as coffee shops, restaurants Asian Thai, wine stores, gym and food shops
* as a reference, I have included a map of venues near current residence in Singapore.

## Business Problem:
The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.
## Interested Audience
I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

In [10]:
# A description of the data and how it will be used to solve the problem. (15 marks)

In [22]:
CLIENT_ID = 'QAADCTQA2LZGYHWATHJT2QDM0FPRRVGT5FCRDOLFUDRT1I45' # your Foursquare ID
CLIENT_SECRET = 'E1I4BB4M0DLJZYQ4KODKAJG1RPJEZTDTOKOMPEHTZL2Q42WB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QAADCTQA2LZGYHWATHJT2QDM0FPRRVGT5FCRDOLFUDRT1I45
CLIENT_SECRET:E1I4BB4M0DLJZYQ4KODKAJG1RPJEZTDTOKOMPEHTZL2Q42WB


In [31]:
# Shenton Way, District 01, Singapore
address = 'Mccallum Street, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore home are {}, {}.'.format(latitude, longitude))

C:\Users\Nageswar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Singapore home are 1.2797332, 103.8472655.


In [24]:
neighborhood_latitude=1.2792655
neighborhood_longitude=103.8480938

In [25]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QAADCTQA2LZGYHWATHJT2QDM0FPRRVGT5FCRDOLFUDRT1I45&client_secret=E1I4BB4M0DLJZYQ4KODKAJG1RPJEZTDTOKOMPEHTZL2Q42WB&v=20180604&ll=1.2792655,103.8480938&radius=500&limit=100'

In [32]:
results = requests.get(url).json()
#results

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Napoleon Food & Wine Bar,Wine Bar,1.279925,103.847333
1,Native,Cocktail Bar,1.280135,103.846844
2,Pepper Bowl,Asian Restaurant,1.279371,103.846710
3,Park Bench Deli,Deli / Bodega,1.279872,103.847287
4,Muchachos,Burrito Place,1.279072,103.847026
5,Mellower Coffee,Café,1.277814,103.848188
6,Dumpling Darlings,Dumpling Restaurant,1.280483,103.846942
7,Sofitel So Singapore,Hotel,1.280124,103.849867
8,Freehouse,Beer Garden,1.281254,103.848513
9,PS.Cafe,Café,1.280468,103.846264


In [35]:
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg